In [80]:
import numpy as np
import pandas as pd

In [81]:
df=pd.read_csv('train.csv')

In [82]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [83]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [84]:
df['title']

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
20795    Rapper T.I.: Trump a ’Poster Child For White S...
20796    N.F.L. Playoffs: Schedule, Matchups and Odds -...
20797    Macy’s Is Said to Receive Takeover Approach by...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799                            What Keeps the F-35 Alive
Name: title, Length: 20800, dtype: object

In [85]:
###Drop Nan Values
df=df.dropna()

In [86]:
df.shape

(18285, 5)

In [87]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [88]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [89]:
## Get the Dependent features
y=df['label']

In [90]:
X.shape

(18285, 4)

In [91]:
y.shape

(18285,)

In [92]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [93]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [94]:
### Vocabulary size
voc_size=5000

## Data Cleaning

In [95]:
messages=X.copy()

In [96]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [97]:
messages.reset_index(inplace=True)

In [98]:
import nltk
import re
from nltk.corpus import stopwords

In [99]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [100]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    # print(i)
    ## Taking only a-zA-Z, removing all symbols and replacing with ''
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    ## Lower all letters
    review = review.lower()
    ## Splitting each word from sentence
    review = review.split()
    ## Removing stopwords and using stemming for dictionary words
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    ## Again joining all words to form a sentence
    review = ' '.join(review)
    ## Appending all the reviews
    corpus.append(review)

In [101]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

## One Hot Representation

In [102]:
## Replacing each word with its index value
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3391, 533, 1915, 4471, 593, 4069, 806, 240, 2487, 3948],
 [565, 4537, 4486, 3457, 103, 4066, 542],
 [3566, 541, 1601, 3140],
 [3476, 1980, 2166, 3023, 2632, 3128],
 [3925, 103, 4540, 998, 798, 1758, 103, 4877, 886, 367],
 [3733,
  2718,
  2688,
  3084,
  2933,
  3703,
  2490,
  2277,
  3458,
  2191,
  3614,
  756,
  4121,
  3367,
  542],
 [2927, 2900, 397, 327, 2646, 2114, 3879, 938, 4992, 1139, 1866],
 [2176, 2631, 3073, 3104, 4078, 4587, 3703, 4858, 4992, 1139, 1866],
 [2561, 3597, 4706, 4385, 1469, 4000, 1811, 3141, 3703, 938],
 [2582, 2026, 2888, 3461, 627, 1439, 4607, 1023],
 [2351, 2440, 4712, 2214, 3594, 4898, 2544, 787, 1158, 3451, 4227],
 [3023, 4178, 593, 4000, 3703, 4078],
 [621, 4239, 2413, 2591, 3876, 522, 4332, 2370, 2370],
 [4075, 2125, 1184, 1281, 541, 4946, 3136, 4992, 1139, 1866],
 [3544, 1759, 1171, 2980, 3678, 4992, 1139, 1866],
 [1298, 3219, 4940, 1995, 4508, 4307, 3147, 1062, 2972, 3484],
 [4082, 1789, 4537],
 [2029, 153, 1772, 2582, 3703, 853, 318, 542],
 [807,

## Embedding Representation

In [103]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  240 2487 3948]
 [   0    0    0 ...  103 4066  542]
 [   0    0    0 ...  541 1601 3140]
 ...
 [   0    0    0 ... 4992 1139 1866]
 [   0    0    0 ...   53 3079 2629]
 [   0    0    0 ... 4041  947 2611]]


In [104]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3391,
        533, 1915, 4471,  593, 4069,  806,  240, 2487, 3948], dtype=int32)

In [105]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_3 (LSTM)               (None, 100)               56400     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [106]:
len(embedded_docs),y.shape

(18285, (18285,))

In [107]:
embedded_docs

array([[   0,    0,    0, ...,  240, 2487, 3948],
       [   0,    0,    0, ...,  103, 4066,  542],
       [   0,    0,    0, ...,  541, 1601, 3140],
       ...,
       [   0,    0,    0, ..., 4992, 1139, 1866],
       [   0,    0,    0, ...,   53, 3079, 2629],
       [   0,    0,    0, ..., 4041,  947, 2611]], dtype=int32)

In [108]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [109]:
X_final

array([[   0,    0,    0, ...,  240, 2487, 3948],
       [   0,    0,    0, ...,  103, 4066,  542],
       [   0,    0,    0, ...,  541, 1601, 3140],
       ...,
       [   0,    0,    0, ..., 4992, 1139, 1866],
       [   0,    0,    0, ...,   53, 3079, 2629],
       [   0,    0,    0, ..., 4041,  947, 2611]], dtype=int32)

In [110]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [111]:
embedded_docs.shape,y.shape

((18285, 20), (18285,))

In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [113]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 37ms/step - loss: 0.3291 - accuracy: 0.8452 - val_loss: 0.1940 - val_accuracy: 0.9180
Epoch 2/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1343 - accuracy: 0.9468 - val_loss: 0.1969 - val_accuracy: 0.9200
Epoch 3/10
192/192 [==============================] - 6s 34ms/step - loss: 0.0946 - accuracy: 0.9657 - val_loss: 0.2218 - val_accuracy: 0.9148
Epoch 4/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0658 - accuracy: 0.9780 - val_loss: 0.2898 - val_accuracy: 0.9165
Epoch 5/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0433 - accuracy: 0.9852 - val_loss: 0.2817 - val_accuracy: 0.9171
Epoch 6/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0278 - accuracy: 0.9911 - val_loss: 0.3845 - val_accuracy: 0.9145
Epoch 7/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0145 - accuracy: 0.9957 - val_loss: 0.4120 - val_accuracy: 0.9163

## Adding Dropout

In [114]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [115]:
### Finally Training
# model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

## Performance Metrics And Accuracy

In [116]:
y_pred=np.argmax(model.predict(X_test),axis=1)

In [117]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]])

In [118]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917